In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [26]:
with open('anna.txt') as f:
    text_raw = f.read()

In [33]:
# Remove legal info from text
splt = text_raw.splitlines()
index = -1

for ind, line in enumerate(splt):
    if "GUTENBERG" in line:
        print(f"Found end of text at line {ind}. \nLine: {line}.")
        index = ind
        break
else:
    print("Didn't find end")

splt = splt[:index-1]
text = "\n".join(splt)

Found end of text at line 39888. 
Line: *** END OF THIS PROJECT GUTENBERG EBOOK ANNA KARENINA ***.


In [118]:
"""
One-hot-encoding of array of integers, array = [0,1,2], num_labels = 3, should return 3x3 identity matrix
"""
def one_hot_encode(array, num_labels):
    out = np.zeros((array.size, num_labels))
    for n in range(0, array.size):
        out[n, array[n]] = 1
    return out

array = np.array([0,1,2])
print(one_hot_encode(array, 3))

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [86]:
"""
From array, generates batches with given batch size and sequence length. 
Returns batches x, y, where y is x offset by 1
"""
def gen_batches(array, batch_size, sequence_length):
    num_per_batch = batch_size*sequence_length
    num_batches= len(array) // num_per_batch
    array = array[:num_batches*num_per_batch]
    array = array.reshape(batch_size, -1)

    for n in range(0, num_batches):
        x = array[:, n*sequence_length:(n+1)*sequence_length]
        y = array[:, n*sequence_length+1:(n+1)*sequence_length+1]
        if len(x[-1]) != len(y[-1]): # Disregard last sequence as num characters differ between x and y
            break
        yield x, y

In [90]:
chars = set(text)
nums = range(0,len(chars))
char2num = dict(zip(chars,nums))
num2char = dict(zip(nums,chars))

<generator object gen_batches at 0x0000025ACF8D0900>